# Notebook 2 — Extraction des embeddings faciaux

## Objectif
Ce notebook a pour but de transformer chaque visage prétraité en un vecteur numérique appelé *embedding facial*.

Ces embeddings sont extraits à l’aide d’un modèle de reconnaissance faciale pré-entraîné.
Aucun entraînement n’est réalisé ici : le modèle est utilisé uniquement en mode *feature extractor*.


In [7]:
from pathlib import Path
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import torch
from torchvision import transforms
from facenet_pytorch import InceptionResnetV1


In [8]:
BASE_DIR = Path("../data")

PROC_FAIRFACE = BASE_DIR / "processed" / "fairface_faces"
PROC_OUR_FACES = BASE_DIR / "processed" / "our_faces"

OUT_DIR = BASE_DIR / "processed"


## Chargement du modèle pré-entraîné

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = InceptionResnetV1(pretrained="vggface2").to(device)
model.eval()

print("Modèle chargé sur :", device)


Modèle chargé sur : cuda


## Prétraitement des images pour le modèle

In [10]:
preprocess = transforms.Compose([
    transforms.Resize((160,160)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5])
])


## Fonction d’extraction d’un embedding

In [11]:
def extract_embedding(image_path):
    img = Image.open(image_path).convert("RGB")
    x = preprocess(img).unsqueeze(0).to(device)

    with torch.no_grad():
        emb = model(x)

    emb = emb.cpu().numpy().flatten()
    emb = emb / np.linalg.norm(emb)  # normalisation L2
    return emb


## Extraction des embeddings FairFace

In [12]:
fairface_embeddings = []
fairface_paths = []

for img_path in PROC_FAIRFACE.glob("*.jpg"):
    emb = extract_embedding(img_path)
    fairface_embeddings.append(emb)
    fairface_paths.append(img_path.name)

fairface_embeddings = np.vstack(fairface_embeddings)


## Extraction des embeddings des photos personnelles

In [13]:
our_embeddings = []
our_meta = []

for person_dir in PROC_OUR_FACES.iterdir():
    if not person_dir.is_dir():
        continue

    for img_path in person_dir.glob("*.jpg"):
        emb = extract_embedding(img_path)
        our_embeddings.append(emb)
        our_meta.append({
            "identity": person_dir.name,
            "filename": img_path.name
        })

our_embeddings = np.vstack(our_embeddings)


## Vérification des dimensions

In [14]:
print("Embeddings FairFace :", fairface_embeddings.shape)
print("Embeddings photos personnelles :", our_embeddings.shape)


Embeddings FairFace : (10530, 512)
Embeddings photos personnelles : (21, 512)


## Sauvegarde des embeddings

In [15]:
np.save(OUT_DIR / "embeddings_fairface.npy", fairface_embeddings)
np.save(OUT_DIR / "embeddings_our_faces.npy", our_embeddings)

print("Embeddings sauvegardés.")


Embeddings sauvegardés.
